In [7]:
import boto3
import botocore
import csv
from io import StringIO
import psycopg2
import pandas as pd

In [8]:
titanic_dataset = pd.read_csv('etl_bites_04_titanic_dataset.csv')

titanic_dataset

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [9]:
# Extract 
def download_csv_from_s3(bucket_name, object_key):
    s3 = boto3.client('s3', config=botocore.config.Config(signature_version=botocore.UNSIGNED))
    response = s3.get_object(Bucket=bucket_name, Key=object_key)
    content = response['Body'].read().decode('utf-8')

    # Process header row to remove leading and trailing spaces
    header, rows = content.split("\n", 1)
    cleaned_header = ",".join(column.strip() for column in header.split(","))

    # Return cleaned content
    return cleaned_header + "\n" + rows



In [10]:
# Transform - Calculate the survival rate for passengers in the 2nd class from the Titanic dataset.
# to calculate survival rate, filter to 2nd class, sum of survived divided by total passengers in second class

def calculate_survival_rate(titanic_data, pclass_filter):
    survived_passengers = 0
    passengers_count = 0

    for row in titanic_data:
        if int(row['Pclass']) == pclass_filter:
            survived_passengers += int(row['Survived'])
            passengers_count += 1
    
    return survived_passengers / passengers_count if passengers_count > 0 else 0


titanic_csv_content = download_csv_from_s3('data-eng-makers-public-datasets-404544469985', 'etl_bites_04_titanic_dataset.csv')

survival_rate_class_2 = calculate_survival_rate(csv.DictReader(StringIO(titanic_csv_content)), 2)


In [11]:
conn_string = "dbname='etl_bites' user='' password='' host='localhost' port='5432'"

In [12]:
def execute_query_postgresql(query):
    with psycopg2.connect(conn_string) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            conn.commit()

new_table_query = """CREATE TABLE class_survival_rate (
    id SERIAL PRIMARY KEY,
    pclass INTEGER NOT NULL,
    survival_rate NUMERIC(10, 5) NOT NULL
);
"""
execute_query_postgresql(new_table_query)

In [13]:
def insert_data_to_postgresql(data, connection):
    cursor = connection.cursor()
    query = "INSERT INTO class_survival_rate (pclass, survival_rate) VALUES (%s, %s)"
    cursor.execute(query, data)
    connection.commit()

conn = psycopg2.connect(conn_string)

insert_data_to_postgresql((2, survival_rate_class_2), conn)